<img style="float:left" width="70%" src="pics/escudo_COLOR_1L_DCHA.png">
<img style="float:right" width="15%" src="pics/PythonLogo.svg">
<br style="clear:both;">


# Juego de *Otelo* **inverso**
### Curso 2022-2023



<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px;">Propuesta</h2>

## Docentes

 - Pedro Latorre Carmona

## Alumno
- Álvaro Manjón Vara

---

# Reglas y objetivos


El juego de **Otelo** se compone de un tablero con forma cuadrada con 8 casillas de lado ($8\times8 = 64$ casillas en total), y 64 fichas con dos caras: Una, negra y otra, blanca.

En el juego de **Otelo inverso**, para esta **segunda convocatoria**, vamos a tener un tablero de $10\times10$, y el objetivo en este caso es terminar con el **menor número de fichas posible**, teniendo en cuenta que siempre hemos de mover, y que se ha de intentar siempre darle la vuelta a una ficha del jugador contrario.

Por tanto, en esta versión del juego:

- Hay que aumentar el tamaño del tablero, a uno de $10\times10$

- Hay que inventarse una heurística que funcione para el nuevo objetivo del juego.

- Jugadores y dinámica restantes, son iguales.

En esta práctica:

- La división/peso de los puntos a la hora de la calificación es la misma que en el caso de la versión original del juego de **Otelo**.

- Todo lo demás también se mantiene igual, salvo por el objetivo a cumplir y el tamaño del tablero.

In [1]:
from copy import deepcopy
from typing import Tuple, List
from IPython.display import clear_output
import time

# Creación del tablero

In [2]:
def crear_tablero(tamaño: int) -> list:
  """
  Crea un tablero vacío con las fichas iniciales de tamaño n x n.
  """
  # Creación del tablero vacío
  tablero = [[0 for i in range(tamaño)] for j in range(tamaño)]

  # Colocación de las fichas iniciales
  primeraCasilla = int(len(tablero)/2 - 1)
  segundaCasilla = int(len(tablero)/2)
  tablero[primeraCasilla][primeraCasilla] = 1
  tablero[primeraCasilla][segundaCasilla] = 2
  tablero[segundaCasilla][primeraCasilla] = 2
  tablero[segundaCasilla][segundaCasilla] = 1
  
  return tablero

## Creación de la clase TableroOtelo

In [3]:
class TableroOtelo:
  
  def __init__(self, matrix):
    self.setMatrix(matrix)
    self.player = 1

    
  # Comprueba si dos objetos de la clase TableroOtelo son iguales
  def __eq__(self, other) -> bool:
    if len(self.matrix) == len(other.matrix):     
      for i in range(self.matrix):
        for j in range(self.matrix):
          if self.matrix[i][j] != other.matrix[i][j]:
            return False
      return True
    return False
      
  # Asigna una matriz como tablero
  def setMatrix(self, matrix):
    self.matrix = deepcopy(matrix)
    
  # Devuelve una copia del tablero
  def getMatrix(self) -> List[List]:
    return deepcopy(self.matrix)
     
  # Muestra el tablero por pantalla
  def showMatrix(self):
    print("  ", end='')
    for i in range(len(self.matrix)):
      print(str(i) + " ", end='')
    print("")  
    print("  ", end='')
    for i in range(len(self.matrix)):
      print("--", end='')
    print("")
    for i in range(len(self.matrix)):
      print(str(i) + "|", end='')
      for j in range(len(self.matrix[i])):
        if self.matrix[i][j] == 0:
          print("- ", end='')
        elif self.matrix[i][j] == 1:
          print("B ", end='')
        elif self.matrix[i][j] == 2:
          print("W ", end='')
        #print(str(self.matrix[i][j]) + " ", end='')
      print()
    
  # Función coste o heurística del juego
  '''
  def utility(self) -> int:
        
    all = [item for sublist in self.matrix for item in sublist]
    white = sum(1 for tile in all if tile == 2)
    black = sum(1 for tile in all if tile == 1)
                
    if white > black:
      diff = -int((white / (black + white)) * 100)     
    else:
      diff = int((black / (black + white)) * 100) 
    
    if self.moveCanBeMade(1) + self.moveCanBeMade(2) == 0:
      mobility = 0    
    else:
      mobility = 100 * self.moveCanBeMade(2) / (self.moveCanBeMade(2) + self.moveCanBeMade(1))
            
    utilidad = diff + mobility
    return utilidad
    '''
  def utility(self) -> int:
    emptyTiles, whiteTiles, blackTiles = self.calculateTiles()
    positionWeights = [
      [30, -5, 15, 12, 10, 10, 12, 15, -5, 30],
      [-5, -10, -6, -4, 2, 2, -4, -6, -10, -5],
      [15, -5, 5, 3, 3, 3, 3, 5, -5, 15],
      [12, -3, 4, 2, 2, 2, 2, 4, -3, 12],
      [10, 2, 3, 1, -1, -1, 1, 3, 2, 10],
      [10, 2, 3, 1, -1, -1, 1, 3, 2, 10],
      [12, -3, 4, 2, 2, 2, 2, 4, -3, 12],
      [15, -5, 5, 3, 3, 3, 3, 5, -5, 15],
      [-5, -10, -6, -4, 2, 2, -4, -6, -10, -5],
      [30, -5, 15, 12, 10, 10, 12, 15, -5, 30]
    ]

    player1Weights = 0
    player2Weights = 0

    for i in range(len(self.matrix)):
      for j in range(len(self.matrix[i])):
        if self.matrix[i][j] == 1:
          player1Weights += positionWeights[i][j]
        elif self.matrix[i][j] == 2:
          player2Weights += positionWeights[i][j]
    player1Score = blackTiles + player1Weights
    player2Score = whiteTiles + player2Weights

    if not(self.moveCanBeMade(1)) and not(self.moveCanBeMade(2)):
      if whiteTiles < blackTiles:
        player2Score -= 1000
      elif whiteTiles > blackTiles:
        player1Score -= 1000

    return player1Score - player2Score

  # Busca todas las piezas que van a cambiar en caso de realizar un movimiento
  def lookForPieces(self, direction: str, firstValue: int, secondValue: int, valRange: int, 
                    playerID: int, element: list, row = 0, col = 0)  -> Tuple[list, bool]:
    changes = []
    searchCompleted = False
    for i in range(firstValue, secondValue, valRange):
      if searchCompleted:
        continue
      piece = element[i]
      if piece == 0:
        changes = []
        searchCompleted = True
      elif piece == playerID:
        searchCompleted = True
      else:
        if direction == "diagonal":
          changes.append((abs(row + (i + 1)), abs(col + (i + 1))))
        else:
          changes.append(i)
    
    return changes, searchCompleted

  # Aplica los cambios en el tablero
  def applyChanges(self, direction: str, changes: list, PLAYMODE: bool, playerID: int, value = 0) -> int:
    if PLAYMODE:
      if direction == "horizontal" or direction == "vertical":
        for i in changes:
          if direction == "horizontal":
            self.matrix[value][i] = playerID
          elif direction == "vertical":
            self.matrix[i][value] = playerID
      elif direction == "diagonal":
        for i,j in changes:
          self.matrix[i][j] = playerID
    return len(changes)

  # Comprueba cuántas fichas van a cambiar al realizar un movimiento, realiza los cambios correspondientes
  def isAvailable(self, row: int, col: int, playerID: int, PLAYMODE:bool) -> int: 

    if PLAYMODE:
      self.matrix[row][col] = playerID
    count = 0
    __column = self.matrix[row]                        
    __row = [self.matrix[i][col] for i in range(0, len(self.matrix))] 
 
    if playerID in __column[:col]:       
      changes, searchCompleted = self.lookForPieces("horizontal", col - 1, -1, -1, playerID, __column) 
      if searchCompleted:       
        count += self.applyChanges("horizontal", changes, PLAYMODE, playerID, row)

    if playerID in __column[col:]:
      changes, searchCompleted = self.lookForPieces("horizontal", col + 1, len(self.matrix), 1, playerID, __column)
      if searchCompleted:       
        count += self.applyChanges("horizontal", changes, PLAYMODE, playerID, row)
    
    if playerID in __row[:row]:                         
      changes, searchCompleted = self.lookForPieces("vertical", row - 1, -1, -1, playerID, __row)
      if searchCompleted:
        count += self.applyChanges("vertical", changes, PLAYMODE, playerID, col)

    if playerID in __row[row:]:
      changes, searchCompleted = self.lookForPieces("vertical", row + 1, len(self.matrix), 1, playerID, __row)
      if searchCompleted:
        count += self.applyChanges("vertical", changes, PLAYMODE, playerID, col)
  
    i = 1    
    ulDiagonal = []
    while row - i >= 0 and col - i >= 0:
      ulDiagonal.append(self.matrix[row - i][col - i])
      i += 1
    if playerID in ulDiagonal:
      changes, searchCompleted = self.lookForPieces("diagonal", 0, len(ulDiagonal), 1, playerID, ulDiagonal, -row, -col)  
      if searchCompleted:
        count += self.applyChanges("diagonal", changes, PLAYMODE, playerID)

    i = 1
    urDiagonal = []
    while row + i < len(self.matrix) and col - i >= 0:
      urDiagonal.append(self.matrix[row + i][col - i])
      i += 1
    if playerID in urDiagonal:
      changes, searchCompleted = self.lookForPieces("diagonal", 0, len(urDiagonal), 1, playerID, urDiagonal, row, -col)                                                 
      if searchCompleted:
          count += self.applyChanges("diagonal", changes, PLAYMODE, playerID)

    i = 1
    llDiagonal = []
    while row - i >= 0 and col + i < len(self.matrix):
      llDiagonal.append(self.matrix[row - i][col + i])
      i += 1
    if playerID in llDiagonal:
      changes, searchCompleted = self.lookForPieces("diagonal", 0, len(llDiagonal), 1, playerID, llDiagonal, -row, col)                                                                                       
      if searchCompleted:
        count += self.applyChanges("diagonal", changes, PLAYMODE, playerID)

    i = 1
    lrDiagonal = []
    while row + i < len(self.matrix) and col + i < len(self.matrix):
      lrDiagonal.append(self.matrix[row + i][col + i])
      i += 1
    if playerID in lrDiagonal:
      changes, searchCompleted = self.lookForPieces("diagonal", 0, len(lrDiagonal), 1, playerID, lrDiagonal, row, col)                                                      
      if searchCompleted:
        count += self.applyChanges("diagonal", changes, PLAYMODE, playerID)

    if count == 0 and PLAYMODE:
      self.matrix[row][col] = 0          

    return count
    
  # Comprueba si el jugador tiene movimientos disponibles  
  def moveCanBeMade(self, playerID:int) -> bool:
    movesFound = False
    for row in range(0, len(self.matrix)):
      for col in range(0, len(self.matrix)):
        if movesFound:
          continue
        elif self.matrix[row][col] == 0:
          numAvailableMoves = self.isAvailable(row, col, playerID, PLAYMODE=False)
          if numAvailableMoves > 0:
            movesFound = True
    
    return movesFound
    
  # Indica quién es el ganador  
  def endGame(self) -> int:
    emptyTiles, whiteTiles, blackTiles = self.calculateTiles()

    if whiteTiles > blackTiles:
      victory = 1
    elif whiteTiles < blackTiles:
      victory = 2
    else:
      victory = -1
    
    return victory
    
  def hasGameEnded(self) -> bool:
    emptyTiles, whiteTiles, blackTiles = self.calculateTiles()
    if emptyTiles == 0 or (whiteTiles == 0 or blackTiles == 0) or (not(self.moveCanBeMade(1)) and not(self.moveCanBeMade(2))):
      return True
    else:
      return False

  def changeTurn(self):
    self.player = 3 - self.player

  def getTurn(self):
    return self.player

  def calculateTiles(self):
    allTiles = [item for sublist in self.matrix for item in sublist]
    emptyTiles = sum(1 for tile in allTiles if tile == 0)
    whiteTiles = sum(1 for tile in allTiles if tile == 2)
    blackTiles = sum(1 for tile in allTiles if tile == 1)

    return emptyTiles, whiteTiles, blackTiles

  def printState(self):
    emptyTiles, whiteTiles, blackTiles = self.calculateTiles()
    print("Estado actual - Vacias: " + str(emptyTiles) + " Blancas: " + str(whiteTiles) + " Negras: " + str(blackTiles))
    print("Turno del jugador " + str(self.getTurn()), end='')
    if self.getTurn() == 1:
      print(" (negras)")
    elif self.getTurn() == 2:
      print(" (blancas)")
    print("")

    
  # Realiza un movimiento  
  def performMove(self,x:int, y:int):
    victory = 0
    changed = False
    if self.matrix[x][y] != 0:   
      print("¡La celda ya está ocupada!")
      print("")
    else:
      numFlipped = self.isAvailable(x, y, self.getTurn(), PLAYMODE=True) 
      if numFlipped == 0:
        print("¡No se puede realizar el movimiento!")
        print("")
      else:
        print("¡Se dieron la vuelta " + str(numFlipped) + " ficha(s)!")
        print("")
        changed = True
        self.changeTurn()
      
        if self.hasGameEnded():
          if not(self.moveCanBeMade(1)) and not(self.moveCanBeMade(2)):
            print("¡Ni el jugador 1 ni el 2 se pueden mover!")
            print("")
          victory = self.endGame()
          return victory, changed
             
        if not(self.moveCanBeMade(self.getTurn())):  
          print("¡El jugador " + str(self.getTurn()) + " no puede mover!")
          print("")
          self.changeTurn()

    return victory, changed

In [4]:
def miniMax(state: TableroOtelo, depth: int, alpha: int, beta: int, nextPlayer: int):

  positions = []

  if (depth == 0 or not state.moveCanBeMade(nextPlayer)):
    return state.getMatrix(), state.utility()

  for row in range(0, len(state.getMatrix())):
    for col in range(0, len(state.getMatrix()[row])):
      if state.getMatrix()[row][col] == 0:
        numAvailableModes = state.isAvailable(row, col, nextPlayer, PLAYMODE=False)
        if numAvailableModes > 0:
          childTable = TableroOtelo(state.getMatrix())
          childTable.isAvailable(row, col, nextPlayer, PLAYMODE=True)
          positions.append(childTable.getMatrix())

  if nextPlayer == 2:
    maxValue = -999
    for child in positions:
      evalMatrix, evalCost = miniMax(TableroOtelo(child), depth - 1, alpha, beta, 3 - nextPlayer)
      if evalCost > maxValue:
        maxValue = evalCost
        bestMatrixMax = child
      alpha = max(alpha, evalCost)
      if beta <= alpha:
        break
    return bestMatrixMax, maxValue
  else:
    minValue = 999
    for child in positions:
      evalMatrix, evalCost = miniMax(TableroOtelo(child), depth - 1, alpha, beta, 3 - nextPlayer)
      if evalCost < minValue:
        minValue = evalCost
        bestMatrixMin = child
      beta = min(beta, evalCost)
      if beta <= alpha:
        break
    return bestMatrixMin, minValue

In [5]:
def performMoveMinMax(state:TableroOtelo, player:int):
  matrizObtenida, coste = miniMax(state, 5, -999, 999, player)
  for row in range(0, len(state.getMatrix())):
    for col in range(0, len(state.getMatrix()[row])):
      if state.getMatrix()[row][col] == 0:
        numAvailableModes = state.isAvailable(row, col, player, PLAYMODE=False)
        if numAvailableModes > 0:
          childTable = TableroOtelo(state.getMatrix())
          childTable.isAvailable(row, col, player, PLAYMODE=True)
          if childTable.getMatrix() == matrizObtenida:
            return row, col

In [6]:
from IPython.display import display
from ipywidgets import HTML

class OteloInverso:
  def __init__(self, tablero: TableroOtelo):
    self.tablero = tablero


  def menu(self):
    entrada = ''
    print("Bienvenido al juego del Otelo inverso")
    print("-----------------------------------")
    print("1. Jugar")
    print("2. Salir")
    while(entrada != 1 and entrada != 2):
      entrada = int(input("Elige una opción: "))
      if entrada == 1:
        self.startGame()
      elif entrada == 2:
        print(" ")
        print("¡Hasta pronto!")
        break
      else:
        print("¡Entrada no válida!")

  def startGame(self):
    entrada = ''
    clear_output(wait=True)
    print("¿Cómo deseas jugar?")
    print("-----------------------------------")
    print("1. Modo texto")
    print("2. Modo gráfico")
    print("3. Salir")
    while(entrada != 1 and entrada != 2 and entrada != 3):
      entrada = int(input("Elige una opción: "))
      if entrada == 1:
        self.entradaMovimientoCLI()
      elif entrada == 2:
        self.entradaMovimientoGUI()
      elif entrada == 3:
        print(" ")
        print("¡Hasta pronto!")
        break
      else:
        print("¡Entrada no válida!")

  def entradaMovimientoCLI(self):
    victoria = 0
    clear_output(wait=True)
    print("Introduce tu primer movimiento")
    print("")
    self.tablero.showMatrix()
    while victoria == 0:
      if self.tablero.getTurn() == 1:
        while True:
          try:
            time.sleep(0.5)
            print("Introduce las coordenadas del movimiento a realizar: ")
            print('(Para salir del juego, introduce en fila o columna "salir")')
            row = input("Fila: ")
            if row == 'salir':
              break
            col = input("Columna: ")
            if col == 'salir':
              break
            row = int(row)
            col = int(col)
            break
          except:
            print("¡Entrada no válida!")
        if row == 'salir' or col == 'salir':
          break
        clear_output(wait=True)
        victoria, changed = self.tablero.performMove(row, col)
        self.tablero.showMatrix()
        if changed:
          print("Movimiento realizado en (" + str(row) + "," + str(col) + ") por jugador " + str(1) + " - negras")
        print("")
        self.tablero.printState()
        print("")
      else:
        print("Esperando movimiento del jugador 2...")
        clear_output(wait=True)
        row, col = performMoveMinMax(self.tablero, 2)
        victoria, changed = self.tablero.performMove(row, col)
        self.tablero.showMatrix()
        if changed:
          print("Movimiento realizado en (" + str(row) + "," + str(col) + ") por jugador " + str(2) + " - blancas")
        print("")
        self.tablero.printState()
        print("")
    print("")
    if victoria == 0:
      print("¡Has salido del juego!")
    elif victoria == 1:
      print("¡Has ganado!")
    elif victoria == 2:
      print("¡Has perdido!")
    elif victoria == -1:
      print("¡Empate!")

  def get_content(self, coord):
    """
    Obtiene el contenido de una determinada posición.
    
    Parameters
    ----------
    coord : Posición [y,x] de la que queremos conocer el contenido
    
    Returns
    --------
    contenido : Una lista de tamaño 1 .
    """
    contenido = [None]
    
    if self.tablero.getMatrix()[coord[0]][coord[1]] == 0:
      contenido[0] = "fondorojo"
    elif self.tablero.getMatrix()[coord[0]][coord[1]] == 1:
      contenido[0] = "caranegrafichafondorojo"
    elif self.tablero.getMatrix()[coord[0]][coord[1]] == 2:
      contenido[0] = "carablancafichafondorojo"
                  
    return contenido
  
  def get_html(self):
    """ Muestra una representación gráfica del juego.

    Devuelve un "string" que contiene HTML

    """ 
    height = len(self.tablero.getMatrix())
    width = len(self.tablero.getMatrix()[0])
    html_string = "<style> img.game {width: 37px !important; height: 22px !important;}</style><table>"

    new_row = "<tr>"
    end_row = "</tr>"
    
    element_image = {
      "fondorojo": "./ImagenesCasillasOtelo/FondoRojo.png",
      "caranegrafichafondorojo": "./ImagenesCasillasOtelo/CaraNegraFichaFondoRojo.png",
      "carablancafichafondorojo": "./ImagenesCasillasOtelo/CaraBlancaFichaFondoRojo.png"
    }

    for i in range(height):
      html_string+=new_row
      for j in range(width):
            
        content = self.get_content((i,j))
        drawing = element_image[content[0]]
            
        html = '<td><img class="game" src=%s alt=""></img></td>' % drawing     
            
            
        html_string+=html
      html_string+=end_row
    html_string += "</table>"
        
                
    return html_string
  
  def showMatrixGUI(self):
    display(HTML(self.get_html()))

  def entradaMovimientoGUI(self):
    victoria = 0
    clear_output(wait=True)
    print("Introduce tu primer movimiento")
    print("")
    self.showMatrixGUI()
    while victoria == 0:
      if self.tablero.getTurn() == 1:
        while True:
          try:
            time.sleep(0.5)
            print("Introduce las coordenadas del movimiento a realizar: ")
            print('(Para salir del juego, introduce en fila o columna "salir")')
            row = input("Fila: ")
            if row == 'salir':
              break
            col = input("Columna: ")
            if col == 'salir':
              break
            row = int(row)
            col = int(col)
            break
          except:
            print("¡Entrada no válida!")
        if row == 'salir' or col == 'salir':
          break
        clear_output(wait=True)
        victoria, changed = self.tablero.performMove(row, col)
        self.showMatrixGUI()
        if changed:
          print("Movimiento realizado en (" + str(row) + "," + str(col) + ") por jugador " + str(1) + " - negras")
        print("")
        self.tablero.printState()
        print("")
      else:
        print("Esperando movimiento del jugador 2...")
        clear_output(wait=True)
        row, col = performMoveMinMax(self.tablero, 2)
        victoria, changed = self.tablero.performMove(row, col)
        self.showMatrixGUI()
        if changed:
          print("Movimiento realizado en (" + str(row) + "," + str(col) + ") por jugador " + str(2) + " - blancas")
        print("")
        self.tablero.printState()
        print("")
    print("")
    if victoria == 0:
      print("¡Has salido del juego!")
    elif victoria == 1:
      print("¡Has ganado!")
    elif victoria == 2:
      print("¡Has perdido!")
    elif victoria == -1:
      print("¡Empate!")



In [7]:
tablero = TableroOtelo(crear_tablero(10))
cli = OteloInverso(tablero)
cli.menu()

¡Se dieron la vuelta 1 ficha(s)!

  0 1 2 3 4 5 6 7 8 9 
  --------------------
0|- - - - - - - - - - 
1|- - - - - - - - - - 
2|- - - - - - - - - - 
3|- - - - B B W - - - 
4|- - B B B W - - - - 
5|- - B B W W W - - - 
6|- - B B W B B - - - 
7|- - - W - - - - - - 
8|- - W B - - - - - - 
9|- - - - - - - - - - 
Movimiento realizado en (8,2) por jugador 2 - blancas

Estado actual - Vacias: 80 Blancas: 8 Negras: 12
Turno del jugador 1 (negras)


Introduce las coordenadas del movimiento a realizar: 
(Para salir del juego, introduce en fila o columna "salir")
Fila: salir

¡Has salido del juego!
